In [1]:
import numpy as np
import pandas as pd

from types import FunctionType

In [2]:
class Perceptron:
    def __init__(self, n_inputs, learning_rate=0.1):
        self.weights = np.zeros(n_inputs)
        self.bias = 0
        self.learning_rate = learning_rate
    
    def __call__(self, inputs):
        return np.heaviside(self.weights @ inputs + self.bias, 0)
    
    def backward(self, inputs, error):
        self.weights += self.learning_rate * error * inputs
        self.bias += self.learning_rate * error

In [3]:
def train_perceptron(perceptron: Perceptron, 
                     inputs: np.ndarray, outputs: np.ndarray) -> None:
    for input, output in zip(inputs, outputs):
        pred = perceptron(input)
        error = output - pred    
        perceptron.backward(input, error)

def test_perceptron(perceptron: Perceptron, inputs: 
                    np.ndarray, outputs: np.ndarray) -> None:
    for input, output in zip(inputs, outputs):
        pred = perceptron(input)
        print(f"{input} -> {output}\nPredicted -> {pred}")

In [4]:
inputs = np.array([[2, 3], [-1, 2]])
outputs = np.array([1, 0])

learning_rate = [0.1, 0.01, 0.001]

for lr in learning_rate:
    perceptron = Perceptron(inputs.shape[1], learning_rate=lr)
    print(f"Learning rate: {lr}")
    train_perceptron(perceptron, inputs, outputs)
    test_perceptron(perceptron, inputs, outputs)
    print()

Learning rate: 0.1
[2 3] -> 1
Predicted -> 1.0
[-1  2] -> 0
Predicted -> 0.0

Learning rate: 0.01
[2 3] -> 1
Predicted -> 1.0
[-1  2] -> 0
Predicted -> 0.0

Learning rate: 0.001
[2 3] -> 1
Predicted -> 1.0
[-1  2] -> 0
Predicted -> 0.0



In [111]:
class Neuron:
    def __init__(self, n_inputs: np.ndarray):
        self.weights = np.random.uniform(-0.1, 0.1, size=n_inputs)
        self.bias = np.random.uniform(-0.1, 0.1)
    
    def __call__(self, inputs: np.ndarray) -> np.ndarray:
        return inputs @ self.weights + self.bias
    
    def backward(self, inputs: np.ndarray, 
                 error: np.ndarray,
                 learning_rate: float) -> None:
        dC_dw = inputs.T @ error
        dC_db = np.sum(error)
        self.weights -= learning_rate * dC_dw
        self.bias -= learning_rate * dC_db

In [116]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return (2 / (1 + np.exp(-2 * x))) - 1

def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

def swish(x):
    return x * sigmoid(x)

In [8]:
def mean_squared_error(error: float) -> float:
    return np.mean(error**2)  

In [86]:
def train(model: Neuron, inputs: np.ndarray, 
          outputs: np.ndarray, epochs: int=100, 
          lr: float=0.1, activation: FunctionType=relu) -> None:

    avg_loss = []
    for _ in range(epochs):
        loss = 0
        pred = activation(model(inputs))
        error = pred - outputs
        model.backward(inputs, error, lr)
        loss += mean_squared_error(error)
        avg_loss.append(loss)

    return round(sum(avg_loss)/epochs, 4), round(avg_loss[-1], 4)        

In [97]:
def predict(model: Neuron, inputs: np.ndarray, 
            activation: FunctionType) -> None:
    return np.round(activation(model(inputs)), 2) 

In [138]:
EPOCH = 1000
LEARNING_RATE = 0.1

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

y_and = np.array([0, 0, 0, 1])
y_or = np.array([0, 1, 1, 1])

model = Neuron(X.shape[1])

avg_loss, final_loss = train(model, X, y_and, epochs=EPOCH, lr=LEARNING_RATE, activation=relu)

print(f"Avg loss -> {avg_loss:.5f}\n")
print(f"Final loss -> {final_loss:.5f}\n")
print(predict(model, X, activation=relu))

Avg loss -> 0.00350

Final loss -> 0.00000

[0. 0. 0. 1.]


In [61]:
def create_csv(data, filename):
    cols = ['Epoch', 'Input', 'Output', 'Predicted', 'Activation', 'Learning Rate', 'Avg Loss', 'Final Loss']
    df = pd.DataFrame(data, columns=cols)
    df.to_csv(filename, index=False, columns=cols, header=cols, sep=',', encoding='utf-8')

In [100]:
def experiments(operator: str):

    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y_and = np.array([0, 0, 0, 1])
    y_or = np.array([0, 1, 1, 1])

    activation_functions = [sigmoid, relu, tanh, gelu, swish]
    epochs = [1, 10, 50, 100, 1000]
    learning_rate = [0.1, 0.01, 0.001]
    results = []

    for i in activation_functions:
        for j in epochs:
            for lr in learning_rate:
                model = Neuron(X.shape[1])
                y = y_and if operator == 'and' else y_or
                avg_loss, final_loss = train(model, X, y, epochs=j, lr=lr, activation=i)
                results.append([j, X.tolist(), y.tolist(), predict(model, X, activation=i), i.__name__, lr, avg_loss, final_loss])
    return results


In [103]:
results_and = experiments('and')
results_or = experiments('or')

create_csv(results_and, 'and.csv')
create_csv(results_or, 'or.csv')

# Wnioski

Dla operatora AND najlepsze rezultaty osiąga funkcja aktywacji ReLU, jest ona jednocześnie najprostszą i dzięki temu najbardziej optymalną wydajnościowo funckją. Najlpeszy Learning Rate wynosi 0.1, przy mniejszych wartościach jak 0.01 lub 0.001, neuron uczy się zdecydowanie wolniej przy 0.01 jest to nie wielka różnica, lecz wzrasta ona zdecydowanie przy 0.001, jak można zauważyć w excelu z wynikami. Opowiednią ilość Epoch dla tej bramki to 1000, co ciekawe można zauważyc, że przy 100 epochcach neuron z learning rate 0.1 radzi sobie bardzo podobnie jak ten z 1000 przy learning rate 0.01. Dzieląc learning rate przez 10 musi pomnożyć wartość epoch o 10 by uzyskać podobny wynik. 

Dla operatora OR najlepiej radzi sobie funkcja aktywacji Tanh przy takim samym learning rate i wartości epoch jak dla bramki AND.

Neuron potrafi także sprawnie nauczyć się bramek AND i OR przy znacznie mniejszej ilości epoch, przykładowo dla bramek OR z funkcja aktywacji tanh i 10 epochami otrzymujemy wyniki [0.5, 0.81, 0.82, 0.94], jest to bardzo bliskie bycia trafną predykcją zakładając, że użyjemy do klasyfikacji funkcji Heaviside(0 gdy x <= 0.5, inaczej 1), wartości dla etykiety 0 są jednak bardzo blisko granicy, co w przypadku testowania na inncyh danych może prowadzić do błędnej klasyfikacji i dużej ilości etykietowania jako false positive, czyli oznaczania 0 jako 1, dlatego lepiej wydłużyć czas uczenia się do przynajmniej 100 epoch dla uzyskania odpowiednich rezultatów, najlepiej by było to 1000 epoch, lecz wtedy trzebą uwazać na tak zwany overfitting, tak by nasz neuron nie zaczął zapamietywać danych uczących.
